In [3]:
from tqdm import tqdm
import pandas as pd
import re
import MeCab
from pykospacing import Spacing
from kss import split_sentences
import torch
from soynlp.normalizer import repeat_normalize

ModuleNotFoundError: No module named 'MeCab'

In [2]:
# GPU 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def remove_emoji(text):
    # 이모지를 정규 표현식으로 제거
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F"  # 감정 이모티콘
        "\U0001F300-\U0001F5FF"  # 기호 및 형태
        "\U0001F680-\U0001F6FF"  # 교통 및 지도 기호
        "\U0001F700-\U0001F77F"  # 연금술 기호
        "\U0001F780-\U0001F7FF"  # 기하학적 도형 확장
        "\U0001F800-\U0001F8FF"  # 보충 화살표-C
        "\U0001F900-\U0001F9FF"  # 보충 기호 및 그림
        "\U0001FA00-\U0001FA6F"  # 체스 기호
        "\U0001FA70-\U0001FAFF"  # 보충 기호 및 그림 확장-A
        "\U00002702-\U000027B0"  # 딩배츠
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", text)  # 이모지 제거 후 반환

In [ ]:
# MeCab 초기화
mecab = MeCab(dicpath="..\mecab\mecab-ko-dic")

# 띄어쓰기 적용을 위한 초기화
spacing = Spacing()

In [5]:
# Step 1: 데이터 로드
file_path = "C:/Users/cho03/Desktop/대학/4학년/2학기/캡스톤디자인-여행사이트/data/lodging_reviews.csv"
data = pd.read_csv(file_path)

# Step 2: 'review_text' 또는 'rating' 컬럼에 NaN이 있는 행 삭제
data = data.dropna(subset=["review_text", "rating"])

# 전처리 함수 정의
pattern = re.compile(f"[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+")
url_pattern = re.compile(
    r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"
)


def clean(x):
    # 텍스트 전처리 함수
    x = pattern.sub(" ", x)  # 특수문자 제거
    x = remove_emoji(x)  # 이모지 삭제
    x = url_pattern.sub("", x)  # URL 제거
    x = x.strip()  # 앞뒤 공백 제거
    x = repeat_normalize(x, num_repeats=2)  # 반복된 글자 정리

    return x


# Step 3: 리뷰 텍스트 전처리
data["review_after"] = data["review_text"].apply(clean)

# Step 4: 리뷰의 길이를 계산하여 'length' 컬럼에 저장
data["length"] = data["review_after"].str.len()

# Step 5: 리뷰의 길이가 10 미만인 데이터 제거
data = data[data["length"] >= 10]

# Step 6: 각 숙소(lodging_id)별 전체 리뷰를 기준으로 평균 평점과 리뷰 개수 계산
lodging_summary = (
    data.groupby("lodging_id")
    .agg(total_review_count=("rating", "count"), average_rating=("rating", "mean"))
    .reset_index()
)

# 조건 1: 전체 리뷰의 평균 평점이 4 이상인 숙소
qualified_lodging_ids = lodging_summary[lodging_summary["average_rating"] >= 4]["lodging_id"]

# Step 7: 조건을 만족하는 숙소 중 평점이 4 이상인 리뷰만 필터링
good_review = data[(data["lodging_id"].isin(qualified_lodging_ids)) & (data["rating"] >= 4)]

# Step 8: 평점이 4 이상인 리뷰 개수가 5개 이상인 숙소만 선택
final_lodging_summary = good_review.groupby("lodging_id").agg(good_review_count=("rating", "count")).reset_index()
final_lodging_ids = final_lodging_summary[final_lodging_summary["good_review_count"] >= 5]["lodging_id"]

# Step 9: 최종 필터링 - 전체 조건을 만족하는 숙소의 평점 4 이상인 리뷰만 추출
filtered_data = good_review[good_review["lodging_id"].isin(final_lodging_ids)]

In [8]:
spaced_reviews = []

# Use tqdm to show progress while applying spacing
for review in tqdm(filtered_data['review_after'], desc="Applying spacing"):
    spaced_reviews.append(spacing(review))

# Assign the spaced reviews back to the DataFrame
filtered_data['review_after'] = spaced_reviews

Applying spacing: 100%|██████████| 8792/8792 [10:06<00:00, 14.49it/s]
C:\Users\cho03\AppData\Local\Temp\ipykernel_18568\2349354966.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['review_after'] = spaced_reviews


In [10]:
filtered_data['review_after']

27       깨끗하고 깔끔합니다. 제주 일주 하면서 이렇게 깨끗한 곳은 처음이네요. 방충망도 꼼...
29       제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다. ...
30       뷰가 일단 대박이구요 룸 컨디션은 쏘 쏘했는데 세탁실 쓸 수 있고 공용 라운지에서 ...
31       제주에 오면 꼭 다시 오고 싶은 곳. 넓고 깔끔 쾌적.세탁기 실도 갖춰 편리. 편백...
32       사장님의 세심함이 느껴지는 게 하. 서울부터 전국여행 중인 부부입니다. 수 많은 게...
                               ...                        
46559    아주 작지만 있을 건 다 있는 정갈한 곳이었다 아침 조식은 12첩 반상 정도로 깔끔...
46561                   찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘 됨
46563    덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...
46564    새 건물처럼 시설 청결하고 방이 무척 따끈해요. 침대도 커서 편해요. 특히 조식 기...
46565    사장님께서 정말 친절하시고, 세심하게 챙겨주셨습니다. 전자레인지 없는 방에 묶었는데...
Name: review_after, Length: 8792, dtype: object

In [11]:
# 'EC' 품사에 의해 문장을 나누는 함수 정의
def split_by_ec(sentences):
    split_sentences = []
    for sentence in sentences:
        # MeCab 파싱
        parsed = mecab.parse(sentence).strip().splitlines()
        current_sentence = []

        for word_info in parsed:
            if word_info == "EOS":  # EOS 무시
                continue
            word, feature = word_info.split("\t")
            pos = feature.split(",")[0]  # 첫 번째 품사 가져오기

            current_sentence.append(word)  # 현재 문장에 단어 추가

            if pos == "EC":
                # 'EC'에서 현재 문장을 나누기
                split_sentences.append("".join(current_sentence).strip())  # 현재 문장 추가
                current_sentence = []  # 현재 문장 초기화

        # 남아있는 문장 추가
        if current_sentence:
            split_sentences.append("".join(current_sentence).strip())

    return split_sentences

In [12]:
# Step 10: 두 가지 방법을 사용하여 문장 분리
final_sentences = []  # 문장 리스트 초기화

for review in tqdm(filtered_data["review_after"], desc="문장 분리 중"):
    # 먼저 kss를 사용하여 문장 분리
    kss_split_sentences = split_sentences(review)
    review_sentences = []  # 현재 리뷰의 문장 저장

    # 그런 다음 각 문장에 대해 split_by_ec를 사용하여 분리
    for sentence in kss_split_sentences:
        ec_split_sentences = split_by_ec([sentence])  # split_by_ec 사용
        # 띄어쓰기 적용
        spaced_sentences = [spacing(s) for s in ec_split_sentences]
        review_sentences.extend(spaced_sentences)  # 리뷰의 문장 리스트에 추가

    final_sentences.append(review_sentences)  # 리뷰의 문장 리스트 추가

문장 분리 중:   0%|          | 0/8792 [00:00<?, ?it/s][Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D

문장 분리 중: 100%|██████████| 8792/8792 [32:33<00:00,  4.50it/s]  


In [13]:
# Step 11: 분리된 리뷰를 filtered_data에 업데이트
filtered_data["review_split"] = final_sentences

C:\Users\cho03\AppData\Local\Temp\ipykernel_18568\2160688168.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["review_split"] = final_sentences


In [14]:
# 최종 데이터 출력
filtered_data

,lodging_id,rating,review_text,photos,date,review_after,length,review_split
27,6,5,깨끗하고 깔끔합니다. 제주일주 하면서 이렇게 깨끗한곳은 처음이네요. 방충망도 꼼꼼하...,NaN,2024.10.25.,깨끗하고 깔끔합니다. 제주 일주 하면서 이렇게 깨끗한 곳은 처음이네요. 방충망도 꼼...,177,"[깨끗하고, 깔끔합니다., 제주 일주하면서, 이렇게 깨끗한 곳은 처음이네요., 방충..."
29,6,5,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다. ...,NaN,2023.02.17.,제가 게스트하우스를 많이 가보지는 않았지만 정말 최고의 숙소라고 할 수 있습니다. ...,158,"[제가 게스트하우스를 많이 가보지, 는 않았지만, 정말 최고의 숙소라고 할 수 있습..."
30,6,4,뷰가 일단 대박이구요 룸 컨디션은 쏘쏘했는데 세탁실 쓸수있고 공용 라운지에서 조식먹...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.10.01.,뷰가 일단 대박이구요 룸 컨디션은 쏘 쏘했는데 세탁실 쓸 수 있고 공용 라운지에서 ...,71,"[뷰가 일단 대박이구요, 룸 컨디션은 쏘쏘했는데, 세탁실 쓸 수 있고, 공용라운지에..."
31,6,5,제주에 오면 꼭 다시 오고 싶은 곳.넓고 깔끔 쾌적.세탁기실도 갖춰 편리.편백나무 ...,['https://img1.kakaocdn.net/cthumb/local/C139x...,2022.07.02.,제주에 오면 꼭 다시 오고 싶은 곳. 넓고 깔끔 쾌적.세탁기 실도 갖춰 편리. 편백...,189,"[제주에 오면, 꼭 다시 오고, 싶은 곳., 넓고, 깔끔 쾌적.세탁기 실도 갖춰 편..."
32,6,5,사장님의 세심함이 느껴지는 게하. 서울부터 전국여행 중인 부부입니다. 수 많은 게하...,NaN,2022.06.26.,사장님의 세심함이 느껴지는 게 하. 서울부터 전국여행 중인 부부입니다. 수 많은 게...,386,"[사장님의 세심함이 느껴지는 게 하., 서울부터 전국여행 중인 부부입니다., 수많은..."
...,...,...,...,...,...,...,...,...
46559,3822,5,아주 작지만 있을건 다있는 정갈한곳이었다\n아침조식은 12첩반상정도로 깔끔한 밥상이...,NaN,2024.04.07.,아주 작지만 있을 건 다 있는 정갈한 곳이었다 아침 조식은 12첩 반상 정도로 깔끔...,78,"[아주 작지만, 있을 건 다 있는 정갈한 곳이었다, 아침 조식은 12첩 반상 정도로..."
46561,3822,5,찐짜 청결함👍👍👍\n방은 작지만 바닥 따뜻하고 \n물 온도도 조절 잘됨,NaN,2023.03.26.,찐짜 청결함 방은 작지만 바닥 따뜻하고 물 온도도 조절 잘 됨,33,"[찐짜청결함방은 작지만, 바닥 따뜻하고, 물 온도도 조절 잘됨]"
46563,3822,5,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,NaN,2023.01.03.,덕유산 등산하기 전 1박한 숙소입니다 긴박하게 잡아 선택의 여지 없어 그냥 간 곳인...,311,"[덕유산 등 산하기 전 1박 한숙소입니다, 긴박하게, 잡아, 선택의 여지없어, 그냥..."
46564,3822,5,새건물처럼 시설청결하고 방이 무척 따끈해요. 침대도 커서 편해요. 특히 조식 기대이...,NaN,2022.01.14.,새 건물처럼 시설 청결하고 방이 무척 따끈해요. 침대도 커서 편해요. 특히 조식 기...,68,"[새 건물처럼 시설 청결하고, 방이 무척 따끈해요., 침대도 커서 편해요., 특히 ..."


In [15]:
# 결과를 CSV 파일로 저장
filtered_data.to_csv("review_split.csv", encoding="utf-8-sig", index=False)